In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download("punkt")
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
files = os.walk("/content/drive/MyDrive/IR_Assignments/IR_assign1")
filepaths = []
for i in files:
  for j in i[2]:
    filepath = i[0] + "/" +j
    filepaths.append(filepath)

In [ ]:
from nltk.stem import PorterStemmer,SnowballStemmer,WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tqdm.notebook import tqdm,tnrange

In [ ]:
db = {}
for i,filepath in enumerate(tqdm(filepaths)):
  try:
    file = open(filepath, encoding="utf8")
    read = file.read().replace('\n', ' ')    
  except Exception as e:
    file = open(filepath, encoding="unicode_escape")
    read = file.read().replace('\n', ' ')
  file.close()
  #stemmer = SnowballStemmer("english")
  stopWords = set(stopwords.words('english'))
  read = read.lower()
  read_tokens = word_tokenize(read)
  preProTokens = [j for j in read_tokens if j not in stopWords]
  preProTokens = [''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation for text in preProTokens]
  preProTokens = [x for x in preProTokens if len(x) > 1]
  tokens = set(preProTokens)
  for k in tokens:
    if k in db:
      db[k].append(i)
    else:
      db[k] = [i]  

  0%|          | 0/1133 [00:00<?, ?it/s]

In [ ]:
from collections import defaultdict
db=defaultdict(lambda:[],db) 

In [ ]:
from tabulate import tabulate

In [ ]:
class Operations():
  def __init__(self,data):
    self.data=set(data)

  def getMapping(self,filepaths):
    data= list(map(lambda i:(i,filepaths[i]),self.data))
    data.sort()      
    print(tabulate(data,headers=['Document ID','Location']))
    
  def __str__(self):
    return str(len(self.data))

  def add(self,operations):
    return Operations(set.union(self.data,operations.data))

  def diff(self,operations):
    return Operations(set.difference(self.data,operations.data))
            
  def intersect(self,operations):
    return Operations(set.intersection(self.data,operations.data))  

In [ ]:
def OR(t1,t2):
  return t1.add(t2)

def AND(t1,t2):
  return t1.intersect(t2)

def ORNOT(t1,t2):
  u = Operations(np.arange(1132))
  not_t2 = u.diff(t2)
  inter = t1.intersect(t2)
  return not_t2.add(inter)

def ANDNOT(t1,t2):
  u = Operations(np.arange(1132))
  not_t2 = u.diff(t2)
  return t1.intersect(not_t2)

In [ ]:
def count(l1,l2):
  i,j,count=0,0,0
  while(i<len(l1) and j<len(l2)):
    count+=1
    if(l1[i]<l2[j]):
      i+=1          
    elif(l1[i]>l2[j]):
      j+=1
    else:
      i+=1
      j+=1
  return count

def OR_comparison(t1,t2):
  l1 = list(t1.data)
  l2 = list(t2.data)
  l1.sort()
  l2.sort()
  return count(l1,l2)

def AND_comparison(t1,t2):
  l1 = list(t1.data)
  l2 = list(t2.data)
  l1.sort()
  l2.sort()
  return count(l1,l2)

def ORNOT_comparison(t1,t2):
  l1 = list(t1.data)
  u = Operations(np.arange(1132))
  not_t2 = u.diff(t2)
  l2 = list(not_t2.data)
  l1.sort()
  l2.sort()
  return count(l1,l2)

def ANDNOT_comparison(t1,t2):
  l1 = list(t1.data)
  u = Operations(np.arange(1132))
  not_t2 = u.diff(t2)
  l2 = list(not_t2.data)
  l1.sort()
  l2.sort()
  return count(l1,l2)

In [ ]:
n = int(input("Enter the number of times to be tested for queries"))
for _ in range(n):
  query = input("Enter input query")
  query = query.lower()
  query_tokens = word_tokenize(query)
  preProTokens = [j for j in query_tokens if j not in stopWords]
  preProTokens = [''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation for text in preProTokens]
  preProTokens = [x for x in preProTokens if len(x) > 1]
  opprand=list(map(str.strip,input("Enter operands").upper().split(",")))
  retrieved = Operations(db[preProTokens[0]])
  comparisons = 0
  for i in tnrange(1,len(preProTokens)):
    current = Operations(db[preProTokens[i]])
    if(opprand[i-1] == 'OR'):
      retrieved = OR(retrieved, current)
      comparisons += OR_comparison(retrieved, current)
    elif(opprand[i-1] == 'AND'):
      retrieved = AND(retrieved, current)
      comparisons += AND_comparison(retrieved, current)
    elif(opprand[i-1] == 'OR NOT'):
      retrieved = ORNOT(retrieved, current)
      comparisons += ORNOT_comparison(retrieved, current)
    elif(opprand[i-1] == 'AND NOT'):
      retrieved = ANDNOT(retrieved, current)
      comparisons += ANDNOT_comparison(retrieved, current)
    else:
      raise Exception("Wrong Operand used "+opprand[i-1])  
  print("Number of documents matched: ",retrieved)
  print("No. of comparisons required: ",comparisons)
  retrieved.getMapping(filepaths)

  

Enter the number of times to be tested for queries1
Enter input querytelephone,paved, roads
Enter operandsOR NOT, AND NOT


  0%|          | 0/2 [00:00<?, ?it/s]

Number of documents matched:  1114
No. of comparisons required:  2246
  Document ID  Location
-------------  --------------------------------------------------------------------
            0  /content/drive/MyDrive/IR_Assignments/IR_assign1/televisi.txt
            1  /content/drive/MyDrive/IR_Assignments/IR_assign1/st_silic.txt
            2  /content/drive/MyDrive/IR_Assignments/IR_assign1/telecom.q
            3  /content/drive/MyDrive/IR_Assignments/IR_assign1/terbear.txt
            4  /content/drive/MyDrive/IR_Assignments/IR_assign1/temphell.jok
            5  /content/drive/MyDrive/IR_Assignments/IR_assign1/televisi.hum
            6  /content/drive/MyDrive/IR_Assignments/IR_assign1/termpoem.txt
            7  /content/drive/MyDrive/IR_Assignments/IR_assign1/subrdead.hum
            8  /content/drive/MyDrive/IR_Assignments/IR_assign1/sysadmin.txt
            9  /content/drive/MyDrive/IR_Assignments/IR_assign1/supermar.rul
           10  /content/drive/MyDrive/IR_Assignments/IR_